# 1D small network

- a network of 20 cells receiving 5 random thalamic inputs

In [1]:
# set fileName

fileName = "eg_small-network"

In [2]:
# set libs

import pyNN.spiNNaker as sim
from pyNN import space 
import numpy as np
import matplotlib.pyplot as plt

Detected PyNN version 0.9.4 and Neo version 0.6.1


In [3]:
# simulation setting

dt         = 1 # ms
min_delay = 1 # ms
max_delay  = 1.0 # ms
simtime    = 1000.0 # ms

sim.setup(timestep = dt, 
          min_dealy = min_delay,
          max_delay = max_delay)

2021-04-23 13:54:08 INFO: Read cfg files: /home/spinnaker/sPyNNaker/lib/python3.6/site-packages/spinn_front_end_common/interface/spinnaker.cfg, /home/spinnaker/sPyNNaker/lib/python3.6/site-packages/spynnaker/pyNN/spynnaker.cfg, /home/spinnaker/.spynnaker.cfg
2021-04-23 13:54:08 INFO: Will search these locations for binaries: /home/spinnaker/sPyNNaker/lib/python3.6/site-packages/spinn_front_end_common/common_model_binaries : /home/spinnaker/sPyNNaker/lib/python3.6/site-packages/spynnaker/pyNN/model_binaries
2021-04-23 13:54:08 WARNING: /home/spinnaker/CNT-2021/SpiNNaker/reports has 8 old reports that have not been closed
2021-04-23 13:54:08 WARNING: /home/spinnaker/CNT-2021/SpiNNaker/application_generated_data_files has 8 old reports that have not been closed
2021-04-23 13:54:08 INFO: Setting time scale factor to 1.
2021-04-23 13:54:08 INFO: Setting machine time step to 1000 micro-seconds.
2021-04-23 13:54:08 WARNING: Extra params {'min_dealy': 1} have been applied to the setup command 

['/home/spinnaker/sPyNNaker/lib/python3.6/site-packages/spinn_front_end_common/interface/spinnaker.cfg', '/home/spinnaker/sPyNNaker/lib/python3.6/site-packages/spynnaker/pyNN/spynnaker.cfg', '/home/spinnaker/.spynnaker.cfg']


0

In [4]:
# network parameters

n_pop = 20     # Number of cells
weight = 0.05  # synaptic weight (µS)

cell_params = {
    'tau_m'      : 20.0,   # (ms)
    'tau_syn_E'  : 2.0,    # (ms)
    'tau_syn_I'  : 4.0,    # (ms)
    'e_rev_E'    : 0.0,    # (mV)
    'e_rev_I'    : -70.0,  # (mV)
    'tau_refrac' : 2.0,    # (ms)
    'v_rest'     : -60.0,  # (mV)
    'v_reset'    : -70.0,  # (mV)
    'v_thresh'   : -50.0,  # (mV)
    'cm'         : 0.5}    # (nF)

# see parameters meaning in http://neuralensemble.org/docs/PyNN/standardmodels.html

In [5]:
# making the network
cells = sim.Population(n_pop, # number of cells
                       sim.IF_cond_exp, # cell model
                       cell_params, # cell parameters
                       label="cells")

# record state variables

cells.record(['spikes'])
cells.record(['v', 'gsyn_exc', 'gsyn_inh'])

In [6]:
# designing the thalamic input

spike_times = [[]]*n_pop #list of spike lists, where one spike list is related to one spike source
random_sources_idx = [np.random.randint(0, n_pop) for i in range(5)]

for idx, sources in enumerate(random_sources_idx):
    #spike_times[sources] = [np.random.normal(loc=10, scale=0.1) for n in range(10)]
    spike_times[sources] = np.sort(np.arange(200, 800, 50).tolist())

spike_source = sim.Population(n_pop, sim.SpikeSourceArray(spike_times=spike_times))
spike_source.record('spikes')

input_conns = sim.Projection(spike_source, # from 
                             cells, # to
                             connector = sim.FixedProbabilityConnector(0.25), 
                             synapse_type = sim.StaticSynapse(weight=weight, delay=min_delay),
                             receptor_type = 'excitatory')

fig, axes = plt.subplots(1,1)
axes_list = fig.axes
axes_list[0].eventplot(spike_times)
axes_list[0].set_xlabel('[ms]')
axes_list[0].set_ylabel('thalamic spike sources')
axes_list[0].set_title('rasterplot')
axes_list[0].set_xlim(0, simtime)
print('selected random sources', random_sources_idx)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

selected random sources [16, 4, 3, 0, 6]


In [ ]:
# run simulation

sim.run(simtime) # 1000 ms


"""
a network of 20 cells receiving 5 random thalamic inputs

"""

2021-04-23 13:54:10 INFO: Simulating for 1000 1.0ms timesteps using a hardware timestep of 1000us
2021-04-23 13:54:10 INFO: Starting execution process
2021-04-23 13:54:13 INFO: Time 0:00:03.172497 taken by SpallocMaxMachineGenerator
Pre allocating resources for Extra Monitor support vertices
|0%                          50%                         100%|
2021-04-23 13:54:22 INFO: Time 0:00:08.465301 taken by PreAllocateResourcesForExtraMonitorSupport
Partitioning graph vertices
|0%                          50%                         100%|
Partitioning graph edges
|0%                          50%                         100%|
2021-04-23 13:54:27 INFO: Time 0:00:04.854196 taken by PartitionAndPlacePartitioner
Created spalloc job 5964556
2021-04-23 13:54:27 INFO: Created spalloc job 5964556
Waiting for board power commands to complete.
2021-04-23 13:54:27 INFO: Waiting for board power commands to complete.
2021-04-23 13:54:32 INFO: Time 0:00:05.047734 taken by SpallocAllocator
2021-04-23 

In [ ]:
# save results

outputs = cells.get_data() # on notebook space
outputs.segments[0].spiketrains


outputs.segments[0]

#cells.write_data(fileName + '.pkl', "spikes") # on folder
#outputs

In [ ]:
# recovering results

def recover_results(outputs):
    results = {}
    
    for analogsignal in outputs.segments[0].analogsignals:
        print(analogsignal.name)
        # to get voltage and conductances
        results[analogsignal.name] = analogsignal
        # to get spikes
        results['spikes'] = outputs.segments[0].spiketrains
    return results

In [ ]:
# check results

results = recover_results(outputs)
results.keys()

In [ ]:
# check the mean firing rate

print("Mean firing rate: ", cells.mean_spike_count() * 1000.0 / simtime, "Hz")

In [ ]:
results['spikes']

In [ ]:
# check the spikes

fig, axes = plt.subplots(1, 1)
fig.tight_layout(pad=5)
axes_list = fig.axes

axes_list[0].eventplot(results['spikes'])
axes_list[0].eventplot(spike_times, color='r')
axes_list[0].set_title('rasterplot of cells')
axes_list[0].set_xlabel('[ms]')
axes_list[0].set_ylabel('cells')
axes_list[0].set_xlim(0, simtime)

# why are there more cells spiking than thalamic source spikes?

In [ ]:
random_sources_idx

In [ ]:
# check the voltage

pickedCell = random_sources_idx[0]

fig, axes = plt.subplots(1, 1)
fig.tight_layout(pad=3)

axes_list = fig.axes
axes_list[0].plot(results['v'].T[pickedCell], label='cell_' + str(pickedCell))

axes_list[0].set_title('voltage of cells')
axes_list[0].set_xlabel('time [ms]')
axes_list[0].set_ylabel('[mV]')
axes_list[0].legend()


In [ ]:
# check the conductance

pickedCell = random_sources_idx[0]

fig, axes = plt.subplots(2, 1)
fig.tight_layout(pad=5)

axes_list = fig.axes

for idx, conductance in enumerate(['gsyn_exc', 'gsyn_inh']):

    fig.tight_layout(pad=3)
    axes_list = fig.axes
    axes_list[idx].plot(results[conductance].T[pickedCell], label='cell_' + str(pickedCell))
    axes_list[idx].set_title(conductance)
    axes_list[idx].set_xlabel('time [ms]')
    axes_list[idx].set_ylabel('[uS]')
    axes_list[idx].legend()

## task 1: on the state variables 

- why the gsyn_inh (conductance related to cell inhibition) is equal to zero?

## task 2: on neuronal parameters

- change the tau_m (membrane time constant) parameter and have a look at the changes on the voltage trace.

- change other neuronal parameters and investigate their effect on the network behaviour

- info about parameters http://neuralensemble.org/docs/PyNN/standardmodels.html

## task 3: on neuronal model

- test another cell type looking at `sim.list_standard_models()`
